# AIFS: Running Inference with Anemoi

This notebook demonstrates how to run the AI-based Forecast System (AIFS) using the Anemoi inference package. You'll learn how to fetch initial conditions from either the latest operational IFS forecast or historical ERA5 data, run the model for a specified lead time, and process the output.

**Notebook Structure:**

1.  **Setup**: Import necessary libraries.
2.  **Configuration**: Define paths, model weights, and data variables.
3.  **Data Fetching**: Functions to get initial conditions from IFS or ERA5.
4.  **Inference**: The main function to run the model.
5.  **Execution**: Run an example inference.
6.  **Visualization**: Code to plot the output data.

## 1. Setup

First, we import all the required libraries. This includes standard libraries like `datetime` and `os`, data handling libraries like `numpy`, `xarray`, and `pandas`, and specialized libraries from `anemoi` and `ecmwf` for running the model and fetching data.

In [1]:
import datetime
import logging
import os
import pickle
import time
import warnings
from collections import defaultdict

import earthkit.data as ekd
import numpy as np
import pandas as pd
import xarray as xr
from anemoi.inference.outputs.printer import print_state
from anemoi.inference.runners.simple import SimpleRunner
from ecmwf.opendata import Client as OpendataClient
from scipy.sparse import load_npz
import gc

warnings.filterwarnings("ignore", category=UserWarning)

logging.basicConfig(
    level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s"
)

## 2. Configuration

This cell sets up the core configuration for the model.

- **`CHECKPOINT`**: Path to the pre-trained AIFS model weights.
- **Regridding Matrices**: Paths to the NumPy sparse matrices (`.npz` files) used to transform data between the standard lat-lon grid and the model's native N320 grid.
- **`ERA5_PATH`**: The Google Cloud Storage path for the ARCO ERA5 dataset, which can be used for historical initial conditions.
- **Constants**: Defines latitudes, longitudes, and the mapping of variable names between the ERA5 dataset and the short names expected by the model for both surface (`SFC_VARS`) and pressure levels (`PL_VARS`).
- **Paths**: Defines and creates local directories to store input and output states, which helps cache data and avoid re-downloading or re-processing.

In [2]:
# Load in the model weights
CHECKPOINT = "weights/AIFS/aifs-single-mse-1.0.ckpt"

# Load in the regridding matrices
LATLON_N320_PATH = "support/regrid/mir_16_linear/9533e90f8433424400ab53c7fafc87ba1a04453093311c0b5bd0b35fedc1fb83.npz"
TFM_LATLON_N320 = load_npz(LATLON_N320_PATH)
N320_LATLON_PATH = "support/regrid/mir_16_linear/7f0be51c7c1f522592c7639e0d3f95bcbff8a044292aa281c1e73b842736d9bf.npz"
TFM_N320_LATLON = load_npz(N320_LATLON_PATH)

# Load in the ERA5 data from Google Cloud Storage
ERA5_PATH = "gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3"
FULL_ERA5 = xr.open_zarr(ERA5_PATH, chunks=None)

# Define the latitude and longitude arrays
LATITUDES = np.linspace(90, -90, 721)
LONGITUDES = np.linspace(0, 359.75, 1440)

# Define the variables for surface and pressure level data
# These dictionaries map the variable names in the ERA5 dataset to their corresponding short names
SFC_VARS = {
    "10m_u_component_of_wind": "10u",
    "10m_v_component_of_wind": "10v",
    "2m_dewpoint_temperature": "2d",
    "2m_temperature": "2t",
    "mean_sea_level_pressure": "msl",
    "skin_temperature": "skt",
    "surface_pressure": "sp",
    "total_column_water": "tcw",
    "land_sea_mask": "lsm",
    "geopotential_at_surface": "z",
    "slope_of_sub_gridscale_orography": "slor",
    "standard_deviation_of_orography": "sdor",
    "soil_temperature_level_1": "stl1",
    "soil_temperature_level_2": "stl2",
    "volumetric_soil_water_layer_1": "swvl1",
    "volumetric_soil_water_layer_2": "swvl2",
}
PL_VARS = {
    "geopotential": "z",
    "temperature": "t",
    "u_component_of_wind": "u",
    "v_component_of_wind": "v",
    "vertical_velocity": "w",
    "specific_humidity": "q",
}
LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]

# Define the input and output state paths
# These paths will be used to save the input and output states of the model
INPUT_STATE_PATH = "input_states"
OUTPUT_STATE_PATH = "output_states"

# Create the directories if they do not exist
for path in [INPUT_STATE_PATH, OUTPUT_STATE_PATH]:
    if not os.path.exists(path):
        os.makedirs(path)

## 3. Data Fetching

We define two functions to prepare the initial conditions for the model: one for fetching the latest operational IFS data and another for using historical ERA5 data.

### 3.1. Get Latest IFS Data

The `get_latest_IFS_data` function fetches the most recent, publicly available forecast data from ECMWF's OpenData service.

- It determines the latest available forecast time.
- It checks if the input state for that time has already been processed and saved locally. If so, it loads the file to save time.
- It downloads the required surface, soil, and pressure level variables for the current time and 6 hours prior.
- The data is transformed from a lat-lon grid to the model's N320 grid using the pre-defined matrix.
- Finally, it saves the processed input state to a `.pkl` file and returns it.

In [3]:
def get_latest_IFS_data() -> tuple[dict | None, datetime.datetime | None]:
    """
    Fetches the latest IFS data from ECMWF OpenData and prepares the input state for model inference.

    Returns:
        tuple: (input_state, DATE) where input_state is a dict containing the processed fields, and DATE is the datetime of the latest data.
    """
    logging.info("Using the latest IFS data from ECMWF OpenData")

    # Surface, soil, and pressure level parameters for IFS
    IFS_PARAM_SFC = [
        "10u",
        "10v",
        "2d",
        "2t",
        "msl",
        "skt",
        "sp",
        "tcw",
        "lsm",
        "z",
        "slor",
        "sdor",
    ]
    IFS_PARAM_SOIL = ["vsw", "sot"]
    IFS_PARAM_PL = ["gh", "t", "u", "v", "w", "q"]
    IFS_LEVELS = [1000, 925, 850, 700, 600, 500, 400, 300, 250, 200, 150, 100, 50]
    IFS_SOIL_LEVELS = [1, 2]

    try:
        DATE = OpendataClient().latest()
    except Exception as e:
        logging.error(f"Error getting latest date: {e}")
        return None, None
    logging.info(f"Initial date is {DATE}")

    save_path: str = (
        f"{INPUT_STATE_PATH}/input_state_{DATE.strftime('%Y%m%dT%H')}_IFS.pkl"
    )
    # Check if the input state already exists
    # If it does, load it instead of fetching new data
    if os.path.exists(save_path):
        logging.info(f"Input state already exists at {save_path}, loading it.")
        with open(save_path, "rb") as f:
            input_state = pickle.load(f)
        return input_state, DATE

    def get_open_data(
        param: list[str], levelist: list[int] = []
    ) -> dict[str, np.ndarray]:
        """
        Helper to fetch and process open data for given parameters and levels.

        Args:
            param: List of parameter names.
            levelist: List of levels (optional).

        Returns:
            dict: Dictionary of processed fields.
        """
        # Initialize a dictionary to hold the fields
        fields = defaultdict(list)
        # Get the data for the current date and the previous date
        for date in [DATE - datetime.timedelta(hours=6), DATE]:
            data = ekd.from_source(
                "ecmwf-open-data", date=date, param=param, levelist=levelist
            )
            for f in data:  # type: ignore
                # Open data is between -180 and 180, we need to shift it to 0-360
                assert f.to_numpy().shape == (721, 1440)
                values = np.roll(f.to_numpy(), -f.shape[1] // 2, axis=1)
                # Interpolate the data to from 0.25 to N320
                # values = ekr.interpolate(
                #     values, {"grid": (0.25, 0.25)}, {"grid": "N320"}
                # )
                values = values.flatten()
                values = TFM_LATLON_N320 * values
                # Add the values to the list
                name = (
                    f"{f.metadata('param')}_{f.metadata('levelist')}"
                    if levelist
                    else f.metadata("param")
                )
                fields[name].append(values)

        # Stack values for each parameter
        for param, values in fields.items():
            fields[param] = np.stack(values)

        return fields

    # Create empty fields dictionary
    fields = {}
    # Get the surface parameters
    fields.update(get_open_data(param=IFS_PARAM_SFC))
    # Get the soil parameters
    soil = get_open_data(param=IFS_PARAM_SOIL, levelist=IFS_SOIL_LEVELS)

    # Map the soil parameters to the expected names
    mapping = {"sot_1": "stl1", "sot_2": "stl2", "vsw_1": "swvl1", "vsw_2": "swvl2"}
    for k, v in soil.items():
        fields[mapping[k]] = v

    # Get the pressure level parameters
    fields.update(get_open_data(param=IFS_PARAM_PL, levelist=IFS_LEVELS))

    # Transform GH to Z (geopotential height to geopotential)
    for level in IFS_LEVELS:
        gh = fields.pop(f"gh_{level}")
        fields[f"z_{level}"] = gh * 9.80665

    input_state = dict(date=DATE, fields=fields)

    # Write out the input state to a file with the date in the filename
    with open(save_path, "wb") as f:
        pickle.dump(input_state, f)

    return input_state, DATE


### 3.2. Get ERA5 Data

The `get_ERA5` function prepares initial conditions from the historical ERA5 dataset for a user-specified date.

- It checks if a processed input file for the given date already exists locally.
- It selects data for the specified `init_date` and 6 hours prior from the Zarr store.
- It renames the variables to the short names required by the model.
- It transforms the data from the ERA5 grid to the model's N320 grid.
- The final state is saved to a `.pkl` file and returned.

In [4]:
def get_ERA5(init_date: datetime.datetime) -> dict:
    """Fetches ERA5 data for a given initialization date and prepares the input state for model inference.
    Args:
        init_date (datetime): The initialization date for which to fetch the ERA5 data.
    Returns:
        dict: A dictionary containing the input state with date and fields.
    """
    # Define the surface and pressure level variables for ERA5
    # These variables will be used to select the relevant data from the ERA5 dataset
    # The variables are mapped to shorter names for input into the model

    logging.info(f"Getting ERA5 data for date {init_date}")

    save_path = (
        f"{INPUT_STATE_PATH}/input_state_{init_date.strftime('%Y%m%dT%H')}_ERA5.pkl"
    )
    # Check if the input state already exists
    # If it does, load it instead of fetching new data
    if os.path.exists(save_path):
        logging.info(
            f"Input state for {init_date} already exists locally. Loading from file..."
        )
        with open(save_path, "rb") as f:
            input_state = pickle.load(f)
        logging.info("Input state loaded successfully.")
        return input_state

    # AIFS requires data from 6 hours before the initialization date
    init_date_minus_6 = init_date - datetime.timedelta(hours=6)

    logging.info("Getting pressure level data...")
    pl_ds = (
        FULL_ERA5[list(PL_VARS.keys())]
        .sel(time=[init_date_minus_6, init_date], level=LEVELS)
        .compute()
        .rename(PL_VARS)
    )

    logging.info("Getting surface level data...")
    sfc_ds = (
        FULL_ERA5[list(SFC_VARS.keys())]
        .sel(time=[init_date_minus_6, init_date])
        .compute()
        .rename(SFC_VARS)
    )

    # AIFS model expects data on a N320 grid,
    # so we need to regrid the ERA5 data
    logging.info("Processing surface level data...")
    fields_sfc = defaultdict(list)
    for date in sfc_ds.time:
        sfc_ds_date = sfc_ds.sel(time=date)
        for param in SFC_VARS.values():
            values = sfc_ds_date[param].to_numpy().flatten()
            values = TFM_LATLON_N320 * values
            fields_sfc[param].append(values)

    logging.info("Processing pressure level data...")
    fields_pl = defaultdict(list)
    for date in pl_ds.time:
        pl_ds_date = pl_ds.sel(time=date)
        for param in PL_VARS.values():
            for level in LEVELS:
                values = pl_ds_date[param].sel(level=level).to_numpy().flatten()
                values = TFM_LATLON_N320 * values
                fields_pl[f"{param}_{level}"].append(values)

    logging.info("Making input state...")
    # Combine the surface and pressure level fields into a single dictionary
    # This dictionary will be used as the input state for the model
    fields = {}
    fields.update(fields_sfc)
    fields.update(fields_pl)

    for param, values in fields.items():
        fields[param] = np.stack(values)

    input_state = dict(date=init_date, fields=fields)

    # Save the input state to a file for later use
    logging.info("Saving input state to file...")
    with open(save_path, "wb") as f:
        pickle.dump(input_state, f)
        logging.info(f"Input state saved to {save_path}")

    logging.info(f"Input state for {init_date} created successfully.")
    return input_state

## 4. Inference and Processing

With the data-loading functions in place, we can now define the functions for running the inference and processing the results.

### 4.1. Process a Single Step

The `process_step` function takes the raw output from the model for a single forecast step and makes it usable.

- The model's output is on the N320 grid, so this function uses the inverse regridding matrix (`TFM_N320_LATLON`) to transform the data back to a standard 0.25-degree lat-lon grid.
- It packages the processed data into a clean `xarray.Dataset`, which is easy to work with and plot.

In [6]:
def process_step(output_state: dict, step: int) -> xr.Dataset:
    """Processes the output state of the model for a given step.
    Args:
        output_state (dict): The output state from the model inference.
        step (int): The current step in hours.
    Returns:
        xr.Dataset: An xarray Dataset containing the processed output state.
    """
    data_vars = {}
    # Because the model outputs are in the N320 grid,
    # we need to transform them back to the 0.25 degree grid
    logging.info(f"Processing step {step}")
    for field in output_state["fields"]:
        values = (
            TFM_N320_LATLON * output_state["fields"][field].reshape(-1, 1)
        ).reshape(721, 1440)
        data_vars[field] = (["lat", "lon"], values.astype(np.float32))

    # Create an xarray Dataset with the processed data
    step_ds = xr.Dataset(
        data_vars,
        coords={"lat": LATITUDES, "lon": LONGITUDES},
    )
    # Add step as an index for concatenation later
    step_ds = step_ds.expand_dims("step")
    step_ds["step"] = [int(step)]
    return step_ds

### 4.2. Run Full Inference

The `run_inference` function is the main entry point for running the forecast.

- **Parameters**:
    - `init_date` (`datetime`): The initialization date for the forecast. If `None`, it uses the latest IFS data. Otherwise, it uses ERA5 data for the specified date.
    - `lead_time` (`int`): The forecast length in hours. It must be a multiple of 6.
    - `save_vars` (`list`): A list of specific variable names to save. If `None`, all variables are saved, which can consume a lot of memory for long forecasts.
- **Workflow**:
    1.  Initializes the `SimpleRunner` with the model checkpoint on the GPU.
    2.  Calls the appropriate data fetching function (`get_latest_IFS_data` or `get_ERA5`).
    3.  Iterates through the forecast steps, from 6 hours up to the `lead_time`.
    4.  At each step, it calls `process_step` to transform and store the output.
    5.  Concatenates the results from all steps into a single `xarray.Dataset`.
    6.  Saves the final dataset to a Zarr file for later use.

In [7]:
def run_inference(
    init_date: datetime.datetime = None, lead_time: int = 360, save_vars: list = None
) -> xr.Dataset:
    """Runs the AIFS model inference for a given initialization date and lead time.
    Args:
        init_date (datetime): The initialization date for the model. If None, uses the latest IFS data.
        lead_time (int): The lead time in hours for the model inference. Must be a multiple of 6 and at least 6.
        save_vars (list): List of variable names to save in the output dataset. If None, saves all variables.
    Returns:
        xr.Dataset: An xarray Dataset containing the model output for the specified lead time.
    """
    # Validate the lead time
    # Ensure that the lead time is a multiple of 6 and at least 6 hours
    if lead_time < 6 or lead_time % 6 != 0:
        raise ValueError(
            "Lead time must be a multiple of 6 hours and at least 6 hours."
        )

    # Validate so the memory usage is manageable
    if save_vars is None and lead_time > 120:
        logging.warning(
            "Running this model for more than 120 steps and saving all variables is not recommended."
        )

    # If no initialization date is provided, fetch the latest IFS data
    # If an initialization date is provided, use it to fetch the ERA5 data
    if init_date is None:
        ic_src = "IFS"
        input_state, init_date = get_latest_IFS_data()
    else:
        ic_src = "ERA5"
        input_state = get_ERA5(init_date)

    # Create the GPU runner for the AIFS model
    # This runner will handle the inference process using the model weights
    runner = SimpleRunner(CHECKPOINT, device="cuda")
    current_step = 0
    start_time = time.perf_counter()
    logging.info("Starting the inference session...")
    current_step_time = time.perf_counter()

    # Initialize a list to store the states for each step
    # This will be used to concatenate the results into a single dataset later
    states = []
    for state in runner.run(input_state=input_state, lead_time=lead_time):
        print_state(state)
        current_step += 6
        # Process the output state for the current step
        if save_vars is None:
            # If no specific variables are requested, process all fields
            states.append(process_step(state, current_step))
        else:
            # If specific variables are requested, filter the fields accordingly
            states.append(
                process_step(
                    {
                        "date": state["date"],
                        "fields": {var: state["fields"][var] for var in save_vars},
                        "latitudes": state["latitudes"],
                        "longitudes": state["longitudes"],
                    },
                    current_step,
                )
            )
        # Time tracking for each step
        step_time = time.perf_counter()
        time_taken = step_time - current_step_time
        logging.info(f"Time taken for step {current_step}: {time_taken:.2f} seconds.")
        current_step_time = step_time
    logging.info("Inference session completed.")
    logging.info(
        f"Total time taken for the inference session: {time.perf_counter() - start_time:.2f} seconds."
    )

    # Concatenate all the processed states into a single xarray Dataset
    # This dataset will contain all the model outputs for the specified lead time
    logging.info("Concatenating all steps into a single dataset.")
    ds = xr.concat(states, dim="step")

    # Clean up the dataset and runner
    del states
    del runner

    # Add the time dimension to the dataset
    # The time dimension will be the forecast initialization date
    ds = ds.expand_dims("time")
    ds["time"] = [pd.to_datetime(input_state["date"])]

    # Save the output dataset to a NetCDF file
    # Only save the dataset if it does not already exist
    save_path = f"{OUTPUT_STATE_PATH}/init_{ic_src}_{init_date.strftime('%Y%m%dT%H')}_lead_{lead_time}.nc"
    if os.path.exists(save_path):
        logging.info(f"Output file {save_path} already exists. Skipping save.")
    else:
        logging.info(f"Saving output dataset to {save_path}")
        ds.to_netcdf(save_path)
    gc.collect()  # Clean up memory
    return ds

## 5. Execution Example

Now, let's run a 120-hour forecast.

- We set `init_date=None` to use the latest available IFS data as our initial condition.
- We set `lead_time` to 120 hours (5 days).
- We specify `save_vars` to only keep the `2t` (2-meter temperature) and `tp` (total precipitation) variables in the final output. This is a good practice to manage memory usage.

The model will now run, and you'll see a log of its progress for each 6-hour step.

In [8]:
# initation date formatted as (YYYY, M, D, H)
init_date = datetime.datetime(2023, 6, 30, 0)
lead_time = 360  # in hours, must be a multiple of 6

# Surface variables must be named as "2t", "tp", etc.
# Pressure levels must be named as "z_500", "t_1000", etc.
# See the get_ERA5 function for the full list of variables
save_vars = ["2t", "tp", "z_500"]

# If init_date is None, the latest IFS data will be used
# If init_date is provided, the ERA5 data will be used

init_date_start = datetime.datetime(2023, 6, 30, 0)
init_date_end = datetime.datetime(2023, 7, 2, 0)
current_date = init_date_start
while current_date <= init_date_end:
    logging.info(f"Running inference for initialization date: {current_date}")
    output_ds = run_inference(
        init_date=current_date, lead_time=lead_time, save_vars=save_vars
    )
    current_date += datetime.timedelta(hours=24)

# output_ds = run_inference(init_date=None, lead_time=lead_time, save_vars=save_vars)

2025-08-19 15:48:01,438 - INFO - Running inference for initialization date: 2023-06-30 00:00:00
2025-08-19 15:48:01,439 - INFO - Getting ERA5 data for date 2023-06-30 00:00:00
2025-08-19 15:48:01,440 - INFO - Input state for 2023-06-30 00:00:00 already exists locally. Loading from file...
2025-08-19 15:48:05,571 - INFO - Input state loaded successfully.
2025-08-19 15:48:05,571 - INFO - Using SimpleRunner runner
2025-08-19 15:48:05,572 - INFO - Starting the inference session...
2025-08-19 15:48:05,641 - INFO - Computed constant forcings: before ['cos_latitude', 'cos_longitude', 'sin_latitude', 'sin_longitude'], after ['cos_latitude', 'cos_longitude', 'sin_latitude', 'sin_longitude']
2025-08-19 15:48:05,642 - INFO - Constant computed forcing: ComputedForcings(['cos_latitude', 'cos_longitude', 'sin_latitude', 'sin_longitude'])
2025-08-19 15:48:05,642 - INFO - Dynamic computed forcing: ComputedForcings(['cos_julian_day', 'cos_local_time', 'sin_julian_day', 'sin_local_time', 'insolation'])



😀 date=2023-06-30T06:00:00 latitudes=(542080,) longitudes=(542080,) fields=102

    q_50   shape=(542080,) min=1.06017e-06    max=3.14451e-06   
    t_1000 shape=(542080,) min=231.488        max=319.61        
    v_925  shape=(542080,) min=-37.536        max=47.5312       
    z_850  shape=(542080,) min=8884.35        max=16325.7       
    swvl2  shape=(542080,) min=0              max=0.763018      
    tcc    shape=(542080,) min=0              max=1             



2025-08-19 15:48:20,295 - INFO - Processing step 12
2025-08-19 15:48:20,324 - INFO - Time taken for step 12: 1.76 seconds.
2025-08-19 15:48:20,363 - INFO - Forecasting step 18:00:00 (2023-06-30 18:00:00)



😀 date=2023-06-30T12:00:00 latitudes=(542080,) longitudes=(542080,) fields=102

    q_50   shape=(542080,) min=1.08673e-06    max=3.16788e-06   
    t_1000 shape=(542080,) min=231.643        max=324.686       
    v_925  shape=(542080,) min=-34.2883       max=41.7049       
    z_850  shape=(542080,) min=8776.32        max=16355.4       
    swvl2  shape=(542080,) min=0              max=0.761413      
    tcc    shape=(542080,) min=0              max=1             



2025-08-19 15:48:21,897 - INFO - Processing step 18
2025-08-19 15:48:21,927 - INFO - Time taken for step 18: 1.60 seconds.
2025-08-19 15:48:21,963 - INFO - Forecasting step 1 day, 0:00:00 (2023-07-01 00:00:00)


...

2025-08-19 15:53:32,021 - INFO - Processing step 354
2025-08-19 15:53:32,040 - INFO - Time taken for step 354: 1.64 seconds.
2025-08-19 15:53:32,073 - INFO - Forecasting step 15 days, 0:00:00 (2023-07-17 00:00:00)



😀 date=2023-07-16T18:00:00 latitudes=(542080,) longitudes=(542080,) fields=102

    q_50   shape=(542080,) min=1.14185e-06    max=3.15891e-06   
    t_1000 shape=(542080,) min=231.244        max=320.622       
    v_925  shape=(542080,) min=-27.2046       max=23.6106       
    z_850  shape=(542080,) min=8894.59        max=16606.2       
    swvl2  shape=(542080,) min=0              max=0.730089      
    tcc    shape=(542080,) min=0              max=1             



2025-08-19 15:53:33,657 - INFO - Processing step 360
2025-08-19 15:53:33,677 - INFO - Time taken for step 360: 1.64 seconds.
2025-08-19 15:53:33,680 - INFO - Inference session completed.
2025-08-19 15:53:33,680 - INFO - Total time taken for the inference session: 104.58 seconds.
2025-08-19 15:53:33,680 - INFO - Concatenating all steps into a single dataset.



😀 date=2023-07-17T00:00:00 latitudes=(542080,) longitudes=(542080,) fields=102

    q_50   shape=(542080,) min=1.1808e-06     max=3.16351e-06   
    t_1000 shape=(542080,) min=230.513        max=318.646       
    v_925  shape=(542080,) min=-28.6854       max=25.919        
    z_850  shape=(542080,) min=8846.28        max=16538.6       
    swvl2  shape=(542080,) min=0              max=0.732057      
    tcc    shape=(542080,) min=0              max=1             



2025-08-19 15:53:33,852 - INFO - Output file output_states/init_ERA5_20230702T00_lead_360.nc already exists. Skipping save.


In [8]:
output_ds

<xarray.Dataset> Size: 748MB
Dimensions:  (time: 1, step: 60, lat: 721, lon: 1440)
Coordinates:
  * lat      (lat) float64 6kB 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * lon      (lon) float64 12kB 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * step     (step) int64 480B 6 12 18 24 30 36 42 ... 330 336 342 348 354 360
  * time     (time) datetime64[ns] 8B 2023-07-02
Data variables:
    2t       (time, step, lat, lon) float32 249MB 274.0 274.0 ... 205.7 205.7
    tp       (time, step, lat, lon) float32 249MB 3.317e-05 3.317e-05 ... 0.0
    z_500    (time, step, lat, lon) float32 249MB 5.565e+04 ... 4.709e+04

## 6. Visualization (Optional)

The following cells contain commented-out code to visualize the forecast output using `matplotlib` and `cartopy`. You can uncomment and run them to plot the results.

### 6.1. Plot 2m Temperature on a Regular Grid

This cell plots the 2-meter temperature (`2t`) for each forecast step on a world map. It converts the temperature from Kelvin to Celsius for easier interpretation.

In [9]:
import matplotlib.pyplot as plt
from cartopy import crs as ccrs
import cartopy.feature as cfeature
import ipywidgets as widgets
from IPython.display import display

# Precompute the plot data
plot_ds = output_ds["2t"].isel(time=0) - 273.15
vmin = plot_ds.min().item()
vmax = plot_ds.max().item()
vmax = max(abs(vmin), abs(vmax))
vmin = -vmax

# Convert steps to a sorted list
step_values = sorted(plot_ds.step.values)

def plot_temperature(step):
    plt.figure(figsize=(10, 5))
    ax = plt.axes(projection=ccrs.PlateCarree())

    # Plot data for selected step
    c = ax.pcolormesh(
        plot_ds.lon,
        plot_ds.lat,
        plot_ds.sel(step=step),
        cmap='RdBu_r',
        vmin=vmin,
        vmax=vmax,
        transform=ccrs.PlateCarree(),
    )

    plt.colorbar(c, ax=ax, orientation='vertical', label='Temperature (°C)', pad=0.01, shrink=0.865)

    # Chicago coordinates
    chicago_lon = 180 + 91
    chicago_lat = 41.85

    temp_at_chicago = plot_ds.sel(step=step).sel(lon=chicago_lon, lat=chicago_lat, method="nearest").item()
    temp_at_chicago_f = (temp_at_chicago * 9/5) + 32

    valid_time = pd.to_datetime(output_ds.time.values[0]) + pd.Timedelta(hours=step)
    valid_time = valid_time - pd.Timedelta(hours=5)  # Convert to CST
    valid_time_str = valid_time.strftime('%Y-%m-%d %H:%M:%S')

    plt.title(f"AIFS 2m Temperature at Chicago: {temp_at_chicago:.1f} °C ({temp_at_chicago_f:.1f} °F) on {valid_time_str} CST")
    plt.scatter([chicago_lon], [chicago_lat], color='black', s=10)

    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")

    plt.tight_layout()
    plt.show()

# Create the slider using valid step values
step_slider = widgets.SelectionSlider(
    options=step_values,
    description="Forecast Step:",
    continuous_update=False,
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Display the interactive plot
widgets.interact(plot_temperature, step=step_slider)


interactive(children=(SelectionSlider(continuous_update=False, description='Forecast Step:', layout=Layout(wid…

<function __main__.plot_temperature(step)>